In [4]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

n_qubits = 4

qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))



gate=[cirq.rz, cirq.rx, cirq.rz]
for k in range(25):
    circuit = []
    symbols=[]
    for k in range(len(qubits)):
        for j in range(3):
            symbol="th_"+str(k)+"_"+str(j)
            symbols.append(symbol)
            circuit.append(gate[j](sympy.Symbol(symbol)).on(qubits[k]))
    #for k in range(len(qubits)):
    #    circuit.append(cirq.CNOT(qubits[k], qubits[(k+1)%len(qubits)]))
    circuit = cirq.Circuit(circuit)

    circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
    output = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00001)))(
    circuit_input,
    symbol_names=symbols,
    operators=tfq.convert_to_tensor([observable]),
    initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

    model = tf.keras.Model(inputs=circuit_input, outputs=output)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse')

    tfqcircuit = tfq.convert_to_tensor([circuit])
    qoutput = tf.ones((1, 1))*-2.*n_qubits
    
    model.set_weights([tf.random.uniform((len(symbols),))])
    model.fit(x=tfqcircuit, y=qoutput,validation_steps=None,
              steps_per_epoch=1, batch_size=1, epochs=20, verbose=1, 
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
    del model
    del output
    del circuit
    gc.collect()

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 2s 2s/sample - loss: 60.8052
Epoch 2/20
1/1 [==============================] - 1s 864ms/sample - loss: 60.7795
Epoch 3/20
1/1 [==============================] - 1s 910ms/sample - loss: 60.7544
Epoch 4/20
1/1 [==============================] - 1s 876ms/sample - loss: 60.7298
Epoch 5/20
1/1 [==============================] - 1s 908ms/sample - loss: 60.7057
Epoch 6/20
1/1 [==============================] - 1s 813ms/sample - loss: 60.6823
Epoch 7/20
1/1 [==============================] - 1s 862ms/sample - loss: 60.6594
Epoch 8/20
1/1 [==============================] - 1s 797ms/sample - loss: 60.6372
Epoch 9/20
1/1 [==============================] - 1s 823ms/sample - loss: 60.6157
Epoch 10/20
1/1 [==============================] - 1s 740ms/sample - loss: 60.5948
Epoch 11/20
1/1 [==============================] - 1s 722ms/sample - loss: 60.5745
Epoch 12/20
1/1 [==============================] - 1s 832ms/sample - loss: 60.55

Epoch 20/20
1/1 [==============================] - 1s 891ms/sample - loss: 60.3339
Train on 1 samples
Epoch 1/20
1/1 [==============================] - 2s 2s/sample - loss: 60.8418
Epoch 2/20
1/1 [==============================] - 1s 818ms/sample - loss: 60.8192
Epoch 3/20
1/1 [==============================] - 1s 742ms/sample - loss: 60.7968
Epoch 4/20
1/1 [==============================] - 1s 793ms/sample - loss: 60.7746
Epoch 5/20
1/1 [==============================] - 1s 742ms/sample - loss: 60.7527
Epoch 6/20
1/1 [==============================] - 1s 696ms/sample - loss: 60.7309
Epoch 7/20
1/1 [==============================] - 1s 656ms/sample - loss: 60.7094
Epoch 8/20
1/1 [==============================] - 1s 1s/sample - loss: 60.6881
Epoch 9/20
1/1 [==============================] - 1s 858ms/sample - loss: 60.6671
Epoch 10/20
1/1 [==============================] - 1s 918ms/sample - loss: 60.6462
Epoch 11/20
1/1 [==============================] - 1s 744ms/sample - loss: 60.6255


Epoch 19/20
1/1 [==============================] - 1s 844ms/sample - loss: 60.6111
Epoch 20/20
1/1 [==============================] - 1s 818ms/sample - loss: 60.5869
Train on 1 samples
Epoch 1/20
1/1 [==============================] - 1s 1s/sample - loss: 60.5659
Epoch 2/20
1/1 [==============================] - 1s 824ms/sample - loss: 60.5485
Epoch 3/20
1/1 [==============================] - 1s 723ms/sample - loss: 60.5312
Epoch 4/20
1/1 [==============================] - 1s 818ms/sample - loss: 60.5143
Epoch 5/20
1/1 [==============================] - 1s 751ms/sample - loss: 60.4976
Epoch 6/20
1/1 [==============================] - 1s 875ms/sample - loss: 60.4813
Epoch 7/20
1/1 [==============================] - 1s 896ms/sample - loss: 60.4653
Epoch 8/20
1/1 [==============================] - 1s 722ms/sample - loss: 60.4497
Epoch 9/20
1/1 [==============================] - 1s 691ms/sample - loss: 60.4344
Epoch 10/20
1/1 [==============================] - 1s 952ms/sample - loss: 60.41

Epoch 20/20
1/1 [==============================] - 1s 1s/sample - loss: 60.1946
Train on 1 samples
Epoch 1/20
1/1 [==============================] - 3s 3s/sample - loss: 60.5062
Epoch 2/20
1/1 [==============================] - 1s 991ms/sample - loss: 60.4847
Epoch 3/20
1/1 [==============================] - 1s 1s/sample - loss: 60.4639
Epoch 4/20
1/1 [==============================] - 1s 1s/sample - loss: 60.4440
Epoch 5/20
1/1 [==============================] - 2s 2s/sample - loss: 60.4249
Epoch 6/20
1/1 [==============================] - 1s 1s/sample - loss: 60.4068
Epoch 7/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3894
Epoch 8/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3729
Epoch 9/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3572
Epoch 10/20
1/1 [==============================] - 1s 884ms/sample - loss: 60.3423
Epoch 11/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3281
Epoch 12/20
1/1 [=======

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 2s 2s/sample - loss: 60.7683
Epoch 2/20
1/1 [==============================] - 1s 862ms/sample - loss: 60.7464
Epoch 3/20
1/1 [==============================] - 1s 930ms/sample - loss: 60.7250
Epoch 4/20
1/1 [==============================] - 1s 896ms/sample - loss: 60.7040
Epoch 5/20
1/1 [==============================] - 1s 884ms/sample - loss: 60.6835
Epoch 6/20
1/1 [==============================] - 1s 886ms/sample - loss: 60.6634
Epoch 7/20
1/1 [==============================] - 1s 734ms/sample - loss: 60.6439
Epoch 8/20
1/1 [==============================] - 1s 850ms/sample - loss: 60.6249
Epoch 9/20
1/1 [==============================] - 1s 945ms/sample - loss: 60.6064
Epoch 10/20
1/1 [==============================] - 1s 891ms/sample - loss: 60.5885
Epoch 11/20
1/1 [==============================] - 1s 886ms/sample - loss: 60.5711
Epoch 12/20
1/1 [==============================] - 1s 843ms/sample - loss: 60.55

Epoch 20/20
1/1 [==============================] - 1s 897ms/sample - loss: 60.0660
